In [4]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ali/key.json"

In [1]:
!gcloud config list

[core]
account = khajasabengr@gmail.com
disable_usage_reporting = True
project = khaja-dev

Your active configuration is: [default]


In [6]:
!gcloud projects list

PROJECT_ID            NAME               PROJECT_NUMBER
apply-ds-test-382504  apply-ds-test      289002071544
caramel-graph-380406  My First Project   834380449602
khaja-dev             khaja-dev          855319760601
khaja-pr              khaja-pr           366071156365
khaja-qa              khaja-qa           973817720022
learn-bq-project1     learn-bq-project1  453715965973
tidy-reporter-380406  My First Project   951114477951


In [2]:
!gcloud config set functions/region asia-south1

Updated property [functions/region].


In [7]:
!gcloud config set project khaja-dev

Updated property [core/project].


In [9]:
!gcloud services list

NAME                              TITLE
bigquery.googleapis.com           BigQuery API
bigquerymigration.googleapis.com  BigQuery Migration API
bigquerystorage.googleapis.com    BigQuery Storage API
cloudapis.googleapis.com          Google Cloud APIs
clouddebugger.googleapis.com      Cloud Debugger API
cloudtrace.googleapis.com         Cloud Trace API
datastore.googleapis.com          Cloud Datastore API
logging.googleapis.com            Cloud Logging API
monitoring.googleapis.com         Cloud Monitoring API
secretmanager.googleapis.com      Secret Manager API
servicemanagement.googleapis.com  Service Management API
serviceusage.googleapis.com       Service Usage API
sql-component.googleapis.com      Cloud SQL
storage-api.googleapis.com        Google Cloud Storage JSON API
storage-component.googleapis.com  Cloud Storage
storage.googleapis.com            Cloud Storage API


In [10]:
!gcloud services enable pubsub.googleapis.com

!gcloud services enable cloudfunctions.googleapis.com

!gcloud services enable cloudbuild.googleapis.com

Operation "operations/acf.p2-855319760601-316beab2-9e45-4005-8453-ace60c14d752" finished successfully.


## Function Creation 

In [75]:
%%writefile main.py
from flask import jsonify

def my_http_function(request, context):
    # Extract the request data
    name = request.args.get('name')

    # Construct the response
    message = f"Hello, {name}!"
    response = {'message': message}

    # Return the response as JSON
    return jsonify(response)


Overwriting main.py


In [76]:
%%writefile requirements.txt
flask

Writing requirements.txt


In [77]:
!ls

GCP_BQ.ipynb		  GCP_GCS.ipynb		   README.md  requirements.txt
GCP_CloudFunctions.ipynb  GCP_SecretManager.ipynb  main.py


## Function deployment 

In [78]:
!gcloud functions deploy mymainfun --runtime python39 --trigger-http --allow-unauthenticated --entry-point my_http_function

Deploying function (may take a while - up to 2 minutes)...⠹                    
For Cloud Build Logs, visit: https://console.cloud.google.com/cloud-build/builds;region=asia-south1/2acdc8ef-7d2b-43f3-ae2d-8db428e9647a?project=855319760601
Deploying function (may take a while - up to 2 minutes)...done.                
availableMemoryMb: 256
buildId: 2acdc8ef-7d2b-43f3-ae2d-8db428e9647a
buildName: projects/855319760601/locations/asia-south1/builds/2acdc8ef-7d2b-43f3-ae2d-8db428e9647a
dockerRegistry: CONTAINER_REGISTRY
entryPoint: my_http_function
httpsTrigger:
  securityLevel: SECURE_OPTIONAL
  url: https://asia-south1-khaja-dev.cloudfunctions.net/mymainfun
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/khaja-dev/locations/asia-south1/functions/mymainfun
runtime: python39
serviceAccountEmail: khaja-dev@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/uploads-240554105391.asia-south1.cloudfunctions.appspot.com/08a92c52-998f-4d8c-b

## Function link

In [79]:
!curl https://asia-south1-khaja-dev.cloudfunctions.net/mymainfun

500 Internal Server Error: The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.

In [80]:
!gcloud functions describe mymainfun --gen2 --region asia-south1 --format="value(serviceConfig.uri)"

https://asia-south1-khaja-dev.cloudfunctions.net/mymainfun


In [81]:
!gcloud functions list

NAME       STATE   TRIGGER       REGION       ENVIRONMENT
mymainfun  ACTIVE  HTTP Trigger  asia-south1  1st gen


In [82]:
!gcloud functions describe mymainfun

availableMemoryMb: 256
buildId: 2acdc8ef-7d2b-43f3-ae2d-8db428e9647a
buildName: projects/855319760601/locations/asia-south1/builds/2acdc8ef-7d2b-43f3-ae2d-8db428e9647a
dockerRegistry: CONTAINER_REGISTRY
entryPoint: my_http_function
httpsTrigger:
  securityLevel: SECURE_OPTIONAL
  url: https://asia-south1-khaja-dev.cloudfunctions.net/mymainfun
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/khaja-dev/locations/asia-south1/functions/mymainfun
runtime: python39
serviceAccountEmail: khaja-dev@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/uploads-240554105391.asia-south1.cloudfunctions.appspot.com/08a92c52-998f-4d8c-ba28-d1f0945d2fcb.zip
status: ACTIVE
timeout: 60s
updateTime: '2023-05-06T12:06:38.111Z'
versionId: '6'


In [83]:
!gcloud functions logs read mymainfun

LEVEL  NAME       EXECUTION_ID  TIME_UTC                 LOG
D      mymainfun  9zn6o48cwa0x  2023-05-06 12:08:17.503  Function execution took 3 ms, finished with status: 'crash'
E      mymainfun  9zn6o48cwa0x  2023-05-06 12:08:17.502  Traceback (most recent call last):
                                                           File "/layers/google.python.pip/pip/lib/python3.9/site-packages/flask/app.py", line 2190, in wsgi_app
                                                             response = self.full_dispatch_request()
                                                           File "/layers/google.python.pip/pip/lib/python3.9/site-packages/flask/app.py", line 1486, in full_dispatch_request
                                                             rv = self.handle_user_exception(e)
                                                           File "/layers/google.python.pip/pip/lib/python3.9/site-packages/flask/app.py", line 1484, in full_dispatch_request
                        

In [84]:
!gcloud functions logs read mymainfun | grep context

                                                         TypeError: my_http_function() missing 1 required positional argument: 'context'
                                                         TypeError: my_http_function() missing 1 required positional argument: 'context'
                                                         TypeError: main() missing 1 required positional argument: 'context'
                                                         TypeError: main() missing 1 required positional argument: 'context'
                                                         TypeError: main() missing 1 required positional argument: 'context'


## Delete Functions 

In [39]:
!gcloud functions delete myfunc --region=us-central1 --quiet

Waiting for operation to finish...done.                                        
Deleted [projects/khaja-dev/locations/us-central1/functions/myfunc].


In [41]:
!gcloud functions delete mymainfun --region=asia-south1 --quiet

Waiting for operation to finish...done.                                        
Deleted [projects/khaja-dev/locations/asia-south1/functions/mymainfun].


## Other Function commands